<a href="https://colab.research.google.com/github/mugalan/classical-mechanics-from-a-geometric-point-of-view/blob/main/rigid-body-control/RigidBodyIntinsicEKF_DHSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setup

In [ ]:
import numpy as np
import scipy as sp
from scipy.integrate import odeint
import math
from numpy import linalg
import sympy
from sympy import symbols
from sympy import *

import plotly.graph_objects as go
import plotly.express as px
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from IPython.display import display, Math, Latex

In [ ]:
!pip install --quiet "git+https://github.com/mugalan/classical-mechanics-from-a-geometric-point-of-view.git#egg=rigid-body-sim"
import sims
mr = sims.RigidBodySim()

# The Kalman Filter on $\mathbb{R}^n$

Consider a system modelled by a Markov process
\begin{align}
x_k &= A_{k-1}\,x_{k-1} + G_{k-1}\,w_{k-1}, \qquad\\
y_k & = H_k\,x_k + z_k. \qquad
\end{align}
We assume $p(w_k)=\mathscr{N}(0,\Sigma_p)$ and $p(z_k)=\mathscr{N}(0,\Sigma_m)$, where $\mathscr{N}(\mu,\Sigma)$ denotes a multivariate Gaussian with mean $\mu$ and covariance $\Sigma$.

Let $Y_k$ denote the event corresponding to a realization of $\{y_1,\dots,y_k\}$. Define
\begin{align}
\widehat{x}_k^+ &\triangleq x_k|Y_k,\qquad\\
\widehat{x}_k^- &\triangleq x_k|Y_{k-1},\qquad\\
y_k^- &\triangleq y_k|Y_{k-1}.\\
\end{align}

Assume the model
\begin{align}
\widehat{x}_k^- &= A_{k-1}\,x_{k-1}^+ + G_{k-1}\,w_{k-1}, \qquad\\
y_k^- &= H_k\,\widehat{x}_k^- + z_k. \qquad
\end{align}
If $p(x_0^-)=\mathscr{N}(m_0,P_0)$, then since $p(w_k)=\mathscr{N}(0,\Sigma_p)$, the linear system above implies
$p(x_k|Y_{k-1})=p(\widehat{x}_k^-)=\mathscr{N}(m_k^-,P_k^-)$,
with predicted mean and covariance
\begin{align}
m_k^- &= A_{k-1}\,m_{k-1}, \qquad\\
P_k^- &= A_{k-1}\,P_{k-1}\,A_{k-1}^{T} + G_{k-1}\,\Sigma_p\,G_{k-1}^{T}. \qquad
\end{align}
Here $p(x_k|Y_k)=p(\widehat{x}_k^+)=\mathscr{N}(m_k,P_k)$.  

Furthermore, since $p(z_k)=\mathscr{N}(0,\Sigma_m)$ we have,
\begin{align}
p(y_k|Y_{k-1})=\mathscr{N}\big(H_k m_k^-,\; H_k P_k^- H_k^{T}+\Sigma_m\big),
\end{align}
so the joint distribution is

\begin{align}
p
\left(\begin{matrix}x_k|Y_{k-1}\\ y_k|Y_{k-1}\end{matrix}\right)
&=
\mathscr{N}\left(
\begin{bmatrix}
m_k^- \\
H_k m_k^-
\end{bmatrix},
\begin{bmatrix}
P_k^- & P_k^- H_k^{T} \\
H_k P_k^- & H_k P_k^- H_k^{T} + \Sigma_m
\end{bmatrix}
\right).
\end{align}

Then from the properties of multivariate normals, the conditional distribution $p(x_k|Y_k)$ is
\begin{align}
p(x_k|Y_k)=\mathscr{N}\Big(
m_k^- + P_k^- H_k^{T}\!\big(H_k P_k^- H_k^{T}+\Sigma_m\big)^{-1}\!(y_k - H_k m_k^-),\;
P_k^- - P_k^- H_k^{T}\!\big(H_k P_k^- H_k^{T}+\Sigma_m\big)^{-1}\!H_k P_k^-
\Big). \quad
\end{align}

Thus the updated mean and covariance are
\begin{align}
K_k &\triangleq P_k^- H_k^{T}\!\big(H_k P_k^- H_k^{T}+\Sigma_m\big)^{-1}, \qquad \\
m_k &= m_k^- + K_k\,(y_k - H_k m_k^-), \qquad \\
P_k &= (I - K_k H_k)\,P_k^-. \qquad
\end{align}
Here $\Sigma_p = \mathbb{E}(w_k w_k^{T})$ and $\Sigma_m=\mathbb{E}(z_k z_k^{T})$. Defining the estimation error $e_k \triangleq x_k - m_k$, we have
\begin{align}
e_k &= (I-K_k H_k) A_{k-1} e_{k-1} + (I-K_k H_k) w_{k-1} - K_k z_k,\\
P_k &= (I-K_k H_k)\big(A_{k-1} P_{k-1} A_{k-1}^{T} + \Sigma_p\big).
\end{align}

### SImulation example

In [ ]:
# 2D position-only measurements (p=2), CV model in x & y
def make_cv2d(
    dt=0.1,
    q=1e-2,
    r=0.5,
    x0=(0, 0, 1, 0.5),
    seed=0,
    *,
    use_G=True,
    noise_model="accel_white",
):
    """
    Build a 2D constant-velocity (CV) linear-Gaussian system.

    States: x = [x, y, vx, vy]^T
      A = [[1, 0, dt, 0 ],
           [0, 1, 0 , dt],
           [0, 0, 1 , 0 ],
           [0, 0, 0 , 1 ]]

    Measurements: y = [x, y]^T  (position only)

    Process-noise options
    ---------------------
    - use_G=True, noise_model='accel_white'  (default):
        Uses an explicit G that maps a 2D white acceleration noise (w ~ N(0, q I_2))
        into the state:
            G = [[dt^2/2,     0   ],
                 [   0  ,  dt^2/2],
                 [  dt ,     0   ],
                 [   0 ,    dt   ]]
        Sigma_p = q * I_2   (in w-space)
        → State-space covariance is G Sigma_p G^T
        (This is the common “white-acceleration” CV model.)

    - use_G=False:
        Legacy behavior (no G). We provide the classic state-space Q directly
        corresponding to (velocity random-walk discretization):
            Q_block = [[dt^3/3, dt^2/2],
                       [dt^2/2, dt     ]] * q
        Q = blockdiag(Q_block, Q_block)
        Sigma_p = Q  (already in state space), G=None

    Notes
    -----
    The two variants imply slightly different discrete-time process covariances.
    Pick the one that matches your physical assumption / reference text.
    """
    A = np.array([
        [1, 0, dt,  0],
        [0, 1,  0, dt],
        [0, 0,  1,  0],
        [0, 0,  0,  1],
    ])
    H = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
    ])
    R = np.eye(2) * (r**2)

    if use_G:
        if noise_model != "accel_white":
            raise ValueError("When use_G=True, supported noise_model is only 'accel_white'.")
        # White acceleration injected into vx, vy
        G = np.array([
            [0.5*dt*dt, 0.0       ],
            [0.0      , 0.5*dt*dt],
            [dt       , 0.0       ],
            [0.0      , dt        ],
        ])
        Sigma_p = q * np.eye(2)  # w-space covariance (2x2)
        return sims.LinearGaussianSystemSyms(
            A=A, H=H, Sigma_p=Sigma_p, Sigma_m=R,
            x0=np.asarray(x0, float),
            rng=np.random.default_rng(seed),
            G=G
        )
    else:
        # Legacy/state-space Q (velocity random-walk discretization)
        Q_block = np.array([[dt**3/3, dt**2/2],
                            [dt**2/2, dt      ]], dtype=float) * q
        Q = np.block([
            [Q_block,               np.zeros((2, 2))],
            [np.zeros((2, 2)),      Q_block        ],
        ])
        return sims.LinearGaussianSystemSyms(
            A=A, H=H, Sigma_p=Q, Sigma_m=R,
            x0=np.asarray(x0, float),
            rng=np.random.default_rng(seed),
            G=G
        )

In [ ]:
sys = make_cv2d(dt=0.1, q=1e-2, r=0.5, x0=(0,0, 0.5, -0.2), seed=7)
X2, Y2 = sys.simulate(T=300)
sys.plot_y(Y2, nbins=40, component_labels=["pos_x", "pos_y"])

In [ ]:
# Option A: simulate 300 steps internally, starting from broad prior
M, Yhat= sys.filter_with_kf_and_plot(T=300, m0=np.array([0,0, 0,0]), P0=np.eye(4)*100,
                                      component_labels=["pos_x","pos_y"], show=True)

# Option B: if you already have measurements Y, just pass them:
# X, Y = sys.simulate(T=300)
# M, Yhat = sys.filter_with_kf_and_plot(Y=Y, m0=np.array([0,0, 0,0]), P0=np.eye(4)*100,
#                                       component_labels=["pos_x","pos_y"])

#### 1-D Example

Let the scalar linear–Gaussian model be
\begin{aligned}
x_k &= a\,x_{k-1} + w_{k-1},\qquad w_{k-1}\sim\mathscr N(0,q),\\
y_k &= h\,x_k + z_k,\qquad\;\;\;\; z_k\sim\mathscr N(0,r),
\end{aligned}
with prior $x_0\sim \mathscr{N}(m_0,P_0)$. Define $Y_{k-1}=\{y_1,\dots,y_{k-1}\}$.

Prediction:

\begin{aligned}
m_k^- &= a\,m_{k-1},\\[2pt]
P_k^- &= a^2 P_{k-1} + q.
\end{aligned}

Innovation (measurement) model and variance:
\begin{aligned}
v_k &\triangleq y_k - h\,m_k^-,\\[2pt]
S_k &= h^2 P_k^- + r.
\end{aligned}

Kalman gain:
\begin{align}
K_k = \dfrac{P_k^-\,h}{S_k}.
\end{align}

Update:
\begin{aligned}
m_k &= m_k^- + K_k\,v_k
= m_k^- + \frac{P_k^- h}{S_k}\bigl(y_k - h\,m_k^-\bigr),\\[6pt]
P_k &= (1 - K_k h)\,P_k^-
= \Bigl(1 - \frac{P_k^- h^2}{S_k}\Bigr) P_k^- .
\end{aligned}

Predictive measurement distribution (before seeing y_k):
\begin{align}
p(y_k | Y_{k-1})=\mathscr N\bigl(h\,m_k^-,\; h^2 P_k^- + r\bigr).
\end{align}

Posterior-predictive measurement distribution (after filtering on y_k):
\begin{align}
p(y_k\mid Y_k)=\mathscr N\bigl(h\,m_k,\; h^2 P_k + r\bigr).
\end{align}



In [ ]:
import numpy as np

def make_cv1d(dt: float = 0.1,
              q: float = 1e-2,
              r: float = 1e-1,
              x0=(0.0, 1.0),
              seed: int | None = None) -> sims.LinearGaussianSystemSyms:
    """
    Build a 1D constant-velocity linear Gaussian system:

        x_k = A x_{k-1} + w_{k-1},      w ~ N(0, Q)
        y_k = H x_k       + z_k,        z ~ N(0, R)

    State: x = [position, velocity]^T  (n=2)
    Measurement: y = position (scalar, p=1)

    Parameters
    ----------
    dt : float
        Sampling period Δt.
    q : float
        Continuous white-acceleration noise intensity (process noise scale).
        Discrete-time Q = q * [[dt^3/3, dt^2/2],
                               [dt^2/2, dt     ]].
    r : float
        Measurement noise std. R = [[r^2]] (scalar variance).
    x0 : tuple[float, float]
        Initial state (position, velocity).
    seed : int | None
        Seed for reproducible randomness.

    Returns
    -------
    LinearGaussianSystemSyms
        System with A, H, Sigma_p (Q), Sigma_m (R), and initial state x0.
    """
    A = np.array([[1.0, dt],
                  [0.0, 1.0]], dtype=float)

    # Measure position only (scalar)
    H = np.array([[1.0, 0.0]], dtype=float)  # shape (1,2)

    # Discrete CV process noise covariance (from white-acceleration model)
    Q = q * np.array([[dt**3/3.0, dt**2/2.0],
                      [dt**2/2.0, dt      ]], dtype=float)

    # Measurement noise covariance (scalar)
    R = np.array([[r**2]], dtype=float)

    x0 = np.asarray(x0, dtype=float)
    if x0.shape != (2,):
        raise ValueError(f"`x0` must be shape (2,), got {x0.shape}.")

    rng = np.random.default_rng(seed)
    return sims.LinearGaussianSystemSyms(A=A, H=H, Sigma_p=Q, Sigma_m=R, x0=x0, rng=rng)

# Must have p=1 in your system (scalar measurement). n can be >1.
sys = make_cv1d(dt=0.1, q=5e-2, r=1.0, x0=(0.0, 0.5), seed=7)  # p=1
# Run with simulated Y (T provided)
sys.animate_measurement_gaussians_scalar(T=80, m0=np.zeros(sys.n), P0=np.eye(sys.n)*100,
                                         frame_ms=120, save_html_path=None, show=True)

# Or if you've already collected Y (shape (T,) or (T,1)):
# _, Y = sys.simulate(T=100)
# sys.animate_measurement_gaussians_scalar(Y=Y, m0=np.zeros(sys.n), P0=np.eye(sys.n)*100,
#                                          save_html_path="kf_scalar_y_gaussians.html", auto_play=False)

#Discrete time pre-observers on Lie Groups with time invariant error dynamics


Let $G$ be an $n$-dimensional Lie group with Lie algebra $G$. Let $(g,\zeta)\in G\times G$ and let $\phi:G\times M\to M$ be a left- or right-invariant action on an $m$-dimensional manifold $M$. We consider the kinematic system
\begin{align}
\dot g &= g\cdot \zeta,  \tag{1}\\
y &= \phi_g(\gamma), \tag{2}
\end{align}
where $\zeta(t)\in G$ is a known input and $\gamma\in M$ is a known constant. A standard discrete-time approximation is
\begin{align}
g_k &= g_{k-1}\exp(\Delta T\zeta_{k-1}), \tag{3}
\end{align}
with sampling period $\Delta T$. (This discretization is classical.)



## For left-invariant outputs

Consider the pre-observer
\begin{align}
\widetilde g_k^- &= \widetilde g_{k-1}\exp(\Delta T\zeta_{k-1}), \tag{4}\\
\widetilde g_k &= \widetilde g_k^-\exp{\big(L(y_k,\widetilde y_k^-)\big)}, \tag{5}\\
\widetilde y_k^- &= \phi_{\widetilde g_k}(\gamma), \tag{6}
\end{align}
where $L:M\times M\to G$ is the innovation term. Define $u_{k-1}\triangleq \exp(\Delta T\zeta_{k-1})$, the a priori error $e_k^-\triangleq (\widetilde g_k^-)^{-1}g_k$, and the a posteriori error $e_k\triangleq \widetilde g_k^{-1}g_k$. Then
\begin{align}
e_k^- &= (\widetilde g_k^-)^{-1} g_k = u_{k-1}^{-1}\widetilde g_{k-1}^{-1} g_{k-1}u_{k-1}
= u_{k-1}^{-1}e_{k-1}u_{k-1},\\
e_k &= \widetilde g_k^{-1} g_k = \exp{\big(-L(y_k,\widetilde y_k^-)\big)}e_k^-= \exp{\big(-L(y_k,\widetilde y_k^-)\big)}I_{u_{k-1}^{-1}}e_{k-1}.
\end{align}

If $L$ is $G$-invariant, i.e., $L\big(\phi_g(y_1),\phi_g(y_2)\big)=L(y_1,y_2)$ for all $g\in G$, then using $y_k=\phi_{g_k}(\gamma)$ and $\widetilde y_k=\phi_{\widetilde g_k}(\gamma)$ one obtains $L(y_k,\widetilde y_k)=L\big(\phi_{e_k}(\gamma),\gamma\big)$, so the error dynamics are autonomous (time invariant).



## For right-invariant outputs

Consider the pre-observer
\begin{align}
\widetilde g_k^- &= \widetilde g_{k-1}\exp(\Delta T\zeta_{k-1}), \tag{7}\\
\widetilde g_k &= \exp{\big(L(y_k,\widetilde y_k^-)\big)}\widetilde g_k^-, \tag{8}\\
\widetilde y_k^- &= \phi_{\widetilde g_k}(\gamma). \tag{9}
\end{align}
With $u_{k-1}\triangleq \exp(\Delta T\zeta_{k-1})$ and the right-invariant errors $e_k^-\triangleq g_k(\widetilde g_k^-)^{-1}$ and $e_k\triangleq g_k\widetilde g_k^{-1}$, the error dynamics satisfy
\begin{align}
e_k^- &= g_k(\widetilde g_k^-)^{-1} = g_{k-1}u_{k-1}u_{k-1}^{-1}\widetilde g_{k-1}^{-1} = e_{k-1},\\
e_k &= g_k\widetilde g_k^{-1} = e_k^-\exp{\big(-L(y_k,\widetilde y_k^-)\big)}= e_{k-1}\exp{\big(-L(y_k,\widetilde y_k^-)\big)}.
\end{align}
If $L$ is $G$-invariant, one similarly gets $L(y_k,\widetilde y_k)=L\big(\phi_{e_k}(\gamma),\gamma\big)$, hence the error dynamics are autonomous.



## The IMU+GNSS sensor fusion problem

A representative application is rigid-body orientation/pose estimation with IMU and GNSS. Gyroscopes measure body-frame angular velocity $\Omega$, accelerometers measure $A_m$ (specific force), and GNSS provides $o,\dot o$. A convenient reformulation introduces
\begin{align}
o_s(t)&\triangleq o(t)+\tfrac{1}{2}gt^2 e_3,\qquad\\
v_s(t)&\triangleq \dot o_s(t)=\dot o(t)+g t\,e_3,\qquad\\
R\,A_m&=\ddot o_s(t)=\ddot o(t)+g e_3,
\end{align}
giving the continuous-time model
\begin{align}
\dot R &= R,\widehat\Omega, \tag{21}\\
\dot v_s &= R,A_m, \tag{22}\\
\dot o_s &= v_s, \tag{23}\\
y_o &= o_s, \tag{24}\\
y_v &= v_s. \tag{25}
\end{align}
Defining the homogeneous state
\begin{align}
X &\triangleq\;
\begin{bmatrix}
R & v_s\\[2pt]
0 & 1
\end{bmatrix},\qquad\\
\zeta &\triangleq
\begin{bmatrix}
\widehat\Omega & A_m\\[2pt]
0 & 0
\end{bmatrix},\qquad\\
\gamma_v &\triangleq\;
\begin{bmatrix}
0_{3\times 1}\\[2pt]
1
\end{bmatrix},
\end{align}
the dynamics/output compactly read
\begin{align}
\dot X &= X\,\zeta, \tag{26}\\
y_v &= X\,\gamma_v. \tag{27}
\end{align}

\begin{align}
\dot o_s &= v_s= X\,\gamma_v, \tag{28}\\
y_o &= o_s. \tag{29}
\end{align}

A first-order discretization of $ \dot X = X\zeta $ gives
\begin{align}
X_{k+1} &= X_k \exp{\big(\Delta t\zeta_k\big)}. \tag{30}
\end{align}

With the block structure of $\zeta_k$, the matrix exponential splits as
\begin{align}
\exp{\big(\Delta t\zeta_k\big)}
&=
\begin{bmatrix}
\exp{\big(\Delta t\widehat\Omega_k\big)} & \Delta t A^m_k\\
0 & 1
\end{bmatrix}.
\end{align}

Therefore, the discrete updates in these shifted variables are
\begin{align}
R_{k+1} &= R_k\exp{\big(\Delta t\widehat\Omega_k\big)},\\
v_{s,k+1} &= v_{s,k} + \Delta tR_k,A^m_k,\\
o_{s,k+1} &= o_{s,k} + \Delta tv_{s,k}.
\end{align}

Returning to the original $(o,v)$ variables (with gravity $g$ along $e_3$), one obtains
\begin{align}
R_{k+1} &\approx R_k\exp{\big(\Delta t\widehat\Omega_k\big)},\\
v_{k+1} &= v_k - g\Delta t\,e_3 + \Delta tR_kA^m_k,\\
o_{k+1} &= o_k + \Delta t\,v_k - \frac{g(\Delta t)^2}{2}\,e_3.
\end{align}

Notes: $\widehat\Omega_k$ denotes the skew-symmetric matrix of the angular velocity sample $\Omega_k$, $A^m_k$ is the measured specific force at step $k$, $e_3=(0,0,1)^\top$.


# Intrinsic Extended Kalman Filter on Lie Groups

## Continuous Time EKF

Let $G$ be an $n$-dimensional Lie group with Lie algebra $G$. Let $(g,\zeta)\in G\times G$ and let $\phi:G\times M\to M$ be a left-invariant linear action on an $m$-dimensional vector space $M$ (so, for each $g\in G$, $\phi_g\in \mathrm{GL}(M)$ is linear). We recall the identities
\begin{align}
g\exp(\zeta)g^{-1} &= \exp{\big(\mathrm{Ad}g\cdot \zeta\big)}, \tag{31}\\
\phi_{\exp(\zeta)} &= \exp{\big(T_e\phi \circ \zeta\big)}. \tag{32}
\end{align}
In particular, if $M=G$ and $\phi=\mathrm{Ad}$ then $\mathrm{Ad}\,{\exp(\zeta)}=\exp(\mathrm{ad}\zeta)$.

We consider left-invariant Markov processes $g(t)\in G$ and $y(t)\in M$ given by
\begin{align}
\dot g &= g\cdot\big(\zeta + n_\zeta\big), \tag{33}\\
y &= \phi_{g^{-1}}(\gamma) + n, \tag{34}
\end{align}
where $\zeta(t)$ and $\gamma\in M$ are known, while $n_\zeta(t)$ and $n(t)$ are zero-mean Gaussian white noises with covariances $\Sigma_g$ and $\Sigma_y$, respectively.

Define
\begin{align}
A(t) &\triangleq -\,\mathrm{ad}{\zeta(t)},\qquad\\
H(t) &\triangleq -\,\phi_{\widetilde g^{-1}}\!\big( (T_e\phi \circ \mathrm{Ad}{\widetilde g})\,\eta_e(\gamma)\big),
\end{align}
and consider the intrinsic EKF
\begin{align}
\dot{\widetilde{g}} &= \widetilde{g}\cdot\big(\zeta + K(t)(y-\widetilde{y})\big), \tag{35}\\
\widetilde y &= \phi_{\widetilde{g}^{-1}}(\gamma), \tag{36}
\end{align}
with Riccati and gain
\begin{align}
\dot P &= A^\top P + P A - P H^\top \Sigma_y^{-1} H P + \Sigma_\zeta,\qquad\\
K &= P H^\top \Sigma_y^{-1}.
\end{align}
If $(A(t),H(t))$ is uniformly observable, the above filter ensures convergence in the sense stated below.

\begin{align}
\dot{e}_g
&= e_g \cdot \Big( (I - \mathrm{Ad}_{e_g^{-1}})\zeta
- \mathrm{Ad}_{e_g^{-1}}K(t)y_e
+ n_\zeta \Big) \tag{39}\\
y_e
&= \phi_{\tilde{g}^{-1}}\Big( \big(\phi_{\tilde{g}e_g^{-1}\tilde{g}^{-1}} - I\big)(\gamma) \Big)
+ n \tag{40}
\end{align}


Introduce the log-error $\eta_e$ and the corresponding output error $y_e$. One obtains the exact error dynamics
\begin{align}
\dot\eta_e &= -\mathrm{ad}\zeta\eta_e
-\Big(\tfrac{\mathrm{ad}{\eta_e}}{\exp(\mathrm{ad}{\eta_e})-I}\Big)K(t)y_e
+\Big(\tfrac{\exp(\mathrm{ad}{\eta_e})\mathrm{ad}{\eta_e}}{\exp(\mathrm{ad}{\eta_e})-I}\Big)n_\zeta, \tag{41}\\
y_e &= \phi_{\widetilde g^{-1}}\Big(\big(\exp(-T_e\phi \circ \mathrm{Ad}{\widetilde g}\cdot \eta_e)-I\big)\gamma\Big) + n. \tag{42}
\end{align}
Linearizing for small $|\eta_e|$ gives
\begin{align}
\dot\eta_e &= \big(A(t)-K(t)H(t)\big)\eta_e + n\zeta - K(t)n
+\Big(\tfrac{1}{2}\mathrm{ad}{\eta_e}+O(|\eta_e|^2)\Big)n\zeta + O(|\eta_e|^2)n, \tag{43}\\
y_e &= H(t)\eta_e + n + O(|\eta_e|^2), \tag{44}
\end{align}
so that the mean $m_{\eta_e}(t)=\mathbb{E}[\eta_e(t)]$ and covariance $P(t)=\mathbb{E}[\eta_e\eta_e^\top]$ evolve as
\begin{align}
\dot m_{\eta_e} &= \big(A(t)-K(t)H(t)\big)m_{\eta_e} + \mathbb{E}[O(|\eta_e|^2)], \tag{45}\\
\dot P &= \big(A(t)-K(t)H(t)\big)P + P\big(A(t)-K(t)H(t)\big)^\top
P H^\top \Sigma_y^{-1} H P + \Sigma_\zeta \\
&\quad + \tfrac{1}{4}\mathbb{E}\big(\mathrm{ad}{\eta_e}\Sigma\zeta\mathrm{ad}_{\eta_e}^\top\big)
\mathbb{E}[O(|\eta_e|^3)]. \tag{46}
\end{align}

Under uniform observability and with $K(t)$ the Kalman gain for $(A(t),H(t),\Sigma_p,\Sigma)$, we have $m_{\eta_e}(t)\to 0$ and, writing $\varepsilon\triangleq \eta_e-m_{\eta_e}$ (mean zero, small covariance), the  Kalman filter property ensures that
that the covariance of $\varepsilon(t)$ converges to a constant and is small. Thus since $e_g = e^{\eta_e} = e^{(m_{\eta_e}+\varepsilon)} \approx e^{m_{\eta_e}}e^{\varepsilon}$ we have that
$\lim_{t\to\infty}e^{g(t)} = e^{\varepsilon}$ and hence that
group error converges to
\begin{align}
\lim_{t\to\infty}\widetilde g = g(t)e^{-\varepsilon(t)}. \tag{47}
\end{align}


## Discretized EKF on Lie Groups - Update

We consider a Lie group $G$ with Lie algebra $\mathfrak{g}$ and a left-invariant linear action $\phi:G\times M\to M$ on a vector space $M$ (so each $\phi_g\in \mathrm{GL}(M)$).

Consider the discretized noisy rigid body kinematics
\begin{align}
g_k &= g_{k-1}\exp{\big(\Delta T\zeta_{k-1}\big)},\\
y_k &= \phi_{g_k^{-1}}(\gamma)+z_k,\qquad p(z_k)=\mathscr{N}(0,\Sigma_m)
\end{align}
This is a left invariant system on $G$ with a right invariant output.

Define the pre-observer
\begin{align}
\widetilde g_k^- &= \widetilde g_{k-1}\exp{\left(\Delta T\left(\zeta_{k-1}+w_{k-1}\right)\right)},\qquad p(w_{k-1})=\mathscr{N}(0,\Sigma_p) \tag{4}\\
\widetilde g_k &= \exp{\big(L(y_k,\widetilde y_k^-)\big)}\widetilde g_k^-, \tag{5}\\
\widetilde y_k^- &= \phi_{{\widetilde{g}_k^-}^{-1}}(\gamma), \tag{6}
\end{align}
where $L:M\times M\to G$ is the innovation term. Define $u_{k-1}\triangleq \exp(\Delta T\zeta_{k-1})$, the a priori error $e_k^-\triangleq g_k(\widetilde g_k^-)^{-1}$, and the a posteriori error $e_k\triangleq g_k\widetilde g_k^{-1}$. Then
\begin{align}
e_k^- &= g_k(\widetilde{g}_k^-)^{-1} ,\\
e_k &= g_k\widetilde{g}_k^{-1} =
g_k{\widetilde{g}_k^-}^{-1}\exp{\big(-L(y_k,\widetilde y_k^-)\big)}\\
&=
g_{k-1}\exp{\big(\Delta T\zeta_{k-1}\big)}
\exp{\left(-\Delta T\left(\zeta_{k-1}+w_{k-1}\right)\right)}
\widetilde{g}_{k-1}^{-1}\exp{\big(-L(y_k,\widetilde y_k^-)\big)}\\
&=
g_{k-1}\widetilde{g}_{k-1}^{-1}\mathbb{I}_{\widetilde{g}_{k-1}}
\left(\exp{\big(\Delta T\zeta_{k-1}\big)}
\exp{\left(-\Delta T\left(\zeta_{k-1}+w_{k-1}\right)\right)}\right)\exp{\big(-L(y_k,\widetilde y_k^-)\big)}\\
&=e_{k-1}\mathbb{I}_{\widetilde{g}_{k-1}}
\Big(\exp{\left(\Delta T\zeta_{k-1}\right)}
\exp{\left(-\Delta T\left(\zeta_{k-1}+w_{k-1}\right)\right)}\Big)\exp{\big(-L(y_k,\widetilde y_k^-)\big)}
\end{align}


For the right invariant output map   we also have
$L(y_k,\widetilde{y}_k^-)$ we also have
\begin{align}
L(y_k,\widetilde{y}_k^-)&=
L(\phi_{g_k^{-1}}(\gamma),\phi_{\widetilde g_k^{-1}}(\gamma))=L(\gamma,\phi_{{\widetilde{g}_k^{-}}^{-1}g_k}(\gamma))=L(\gamma,\phi_{e_k^-}(\gamma))
\end{align}

---

Define $e_k=e^{\eta_k}$ and also recall
\begin{align}
\log(\exp X\,\exp Y)=X+\Phi(X)\,Y+O(\|Y\|^2)
\end{align}
with
\begin{align}
\displaystyle \Phi(\eta)=\frac{\mathrm{ad}\eta\,e^{\mathrm{ad}\eta}}{e^{\mathrm{ad}\eta}-I}
= I+\sum_{m=1}^{\infty}\frac{(-1)^{m+1}}{m(m+1)}\big(e^{\mathrm{ad}_\eta}-I\big)^m.
\end{align}

Note that
\begin{align}\Big(\exp{\left(\Delta T\zeta_{k-1}\right)}
\exp{\left(-\Delta T\left(\zeta_{k-1}+w_{k-1}\right)\right)}\Big)&=
\exp{\Big(
\Delta T\zeta_{k-1} - \Delta T\Phi(\Delta T\zeta_{k-1})\left(\zeta_{k-1}+w_{k-1}\right)+O(||\Delta T\zeta_{k-1}+\sqrt{\Delta T}w_{k-1}||^2)   
\Big)}.
\end{align}
Let
\begin{align}
Y&\triangleq
\Big(
\Delta T\zeta_{k-1} - \Delta T\Phi(\Delta T\zeta_{k-1})\left(\zeta_{k-1}+w_{k-1}\right)+O(||\Delta T\zeta_{k-1}+\sqrt{\Delta T}w_{k-1}||^2)    
\Big)
\end{align}

---

Then we have
\begin{align}
\exp{\eta_k} &=  
\exp{(\eta_{k-1})}\mathbb{I}_{\widetilde{g}_{k-1}}\exp{(Y)}\exp{\big(-L(y_k,\widetilde y_k^-)\big)},\\
&=  
\exp{(\eta_{k-1})}\exp{\left(\operatorname{Ad}_{\widetilde{g}_{k-1}}Y\right)}\exp{\big(-L(y_k,\widetilde y_k^-)\big)},\\
&=  
\exp{(\eta_{k-1})}\exp{\left(
\operatorname{Ad}_{\widetilde{g}_{k-1}}Y-\Phi\left(\operatorname{Ad}_{\widetilde{g}_{k-1}}Y\right)L(y_k,\widetilde y_k^-)
+O\left(||L(y_k,\widetilde y_k^-)||^2\right)
\right)},\\
&=\exp{\left(\eta_{k-1}-\Phi(\eta_{k-1})
\left(
\operatorname{Ad}_{\widetilde{g}_{k-1}}Y-\Phi\left(\operatorname{Ad}_{\widetilde{g}_{k-1}}Y\right)L(y_k,\widetilde y_k^-)
+O\left(||L(y_k,\widetilde y_k^-)||^2\right)
\right)+H.O.T.
\right)}
\end{align}

Recalling
\begin{align}
\Phi(\eta) = I + \frac{1}{2} \operatorname{ad}_\eta + \frac{1}{12} \operatorname{ad}_\eta^2 + O(\|\eta\|^3)
\end{align}
and noting that $L(y_k,\widetilde y_k^-)\sim \eta_k$
we have retaining the $\eta_{k-1}$ order terms only
\begin{align}
\eta_k &=  
\eta_{k-1}-\Phi(\eta_{k-1})
\left(
\operatorname{Ad}_{\widetilde{g}_{k-1}}Y-\Phi\left(\operatorname{Ad}_{\widetilde{g}_{k-1}}Y\right)L(y_k,\widetilde y_k^-)
+O\left(||L(y_k,\widetilde y_k^-)||^2\right)
\right)+H.O.T.\\
&=  
\eta_{k-1}-\left(I + \frac{1}{2} \operatorname{ad}_{\eta_{k-1}}\right)
\left(
\operatorname{Ad}_{\widetilde{g}_{k-1}}Y-
\left(I + \frac{1}{2} \operatorname{ad}_{\operatorname{Ad}_{\widetilde{g}_{k-1}}Y}\right)L(y_k,\widetilde y_k^-)
\right)+H.O.T.
\end{align}

Re-arrannging and grouping the $||L||^2$ and $||w||^2$ terms we have
\begin{align}
\eta_k
&=
\eta_{k-1}
+ \Phi(\eta_{k-1})
\Big(
  \Delta T\operatorname{Ad}_{\widetilde g_{k-1}}\Phi(-\Delta T\zeta_{k-1})\,w_{k-1}
  -
  \Phi\!\big(\Delta T\operatorname{Ad}_{\widetilde g_{k-1}}\Phi(-\Delta T\zeta_{k-1})\,w_{k-1}\big)
  L(y_k,\widetilde y_k^-)
\Big)
+ O(\|L\|^2,\|w\|^2).
\end{align}

Neglecting higher-order cross and quadratic terms
$O(\|L\|\|w\|)$, $O(\|L\|^2)$, and $O(\|w\|^2)$,
while keeping the finite-step operator $\Phi(-\Delta T\zeta_{k-1})$ exact,
the error recursion reduces to
\begin{align}
\eta_k
&=
\eta_{k-1}
+ \Phi(\eta_{k-1})
\Big(
  \Delta T\operatorname{Ad}_{\widetilde g_{k-1}}
  \Phi(-\Delta T\zeta_{k-1})\,w_{k-1}
\Big)
- \Phi(\eta_{k-1})\,L(y_k,\widetilde y_k^-)
+ O(\|L\|^2,\|w\|^2).
\end{align}
Here, $\Phi(\eta_{k-1}) = I + \tfrac{1}{2}\operatorname{ad}_{\eta_{k-1}} + O(\|\eta_{k-1}\|^2)$
and $\Phi(-\Delta T\zeta_{k-1})$ is retained in closed form to preserve the exact
finite-step propagation of the process dynamics.


Now lets expnad the output map.
\begin{align}
L(y_k,\widetilde{y}_k^-)&=
L(\phi_{g_k^{-1}}(\gamma),\phi_{\widetilde g_k^{-1}}(\gamma))=L(\gamma,\phi_{{\widetilde{g}_k^{-}}^{-1}g_k}(\gamma))=L(\gamma,\phi_{e_k^-}(\gamma))
\end{align}
\begin{align}
e_k^- &= g_k(\widetilde{g}_k^-)^{-1} =g_{k-1}\exp{\left(\Delta T \zeta_{k-1}\right)}\exp{\left(-\Delta T\left( \zeta_{k-1}+w_{k-1}\right)\right)}\widetilde{g}_{k-1}^{-1}\\
&=e_{k-1}\mathbb{I}_{\widetilde{g}_{k-1}}\exp{\left(\Delta T \zeta_{k-1}\right)}\exp{\left(-\Delta T\left( \zeta_{k-1}+w_{k-1}\right)\right)}\\
&=e_{k-1}\mathbb{I}_{\widetilde{g}_{k-1}}
\exp{\left(\Delta T \zeta_{k-1}
-\Delta T \Phi\left(\Delta T \zeta_{k-1}\right)\left(\zeta_{k-1}+w_{k-1}\right)
+O\left(\Delta T^2||\zeta_{k-1}w_{k-1}||^2\right)\right)}\\
&=e_{k-1}
\exp{\left(\operatorname{Ad}_{\widetilde{g}_{k-1}}\left(\Delta T \zeta_{k-1}
-\Delta T \Phi\left(\Delta T \zeta_{k-1}\right)\left(\zeta_{k-1}+w_{k-1}\right)
+O\left(\Delta T^2 ||\zeta_{k-1}+w_{k-1}||^2\right)\right)\right)}
\end{align}

\begin{align}
L(y_k,\widetilde{y}_k^-)
&= L(\phi_{g_k^{-1}}(\gamma),\phi_{\widetilde g_k^{-1}}(\gamma))
 = L(\gamma,\phi_{{\widetilde g_k^-}^{-1}g_k}(\gamma))
 = L(\gamma,\phi_{e_k^-}(\gamma)),\\[1ex]
e_k^-
&= e_{k-1}\,
   \mathbb{I}_{\widetilde g_{k-1}}\!\left(
   \exp(\Delta T\zeta_{k-1})
   \exp{\left(-\Delta T\left(\zeta_{k-1}+w_{k-1}\right)\right)}
   \right)\\[1ex]
&= e_{k-1}\,
   \exp\!\Big(
   \operatorname{Ad}_{\widetilde g_{k-1}}\!\big[
   \Delta T\zeta_{k-1}
   - \Delta T\Phi(\Delta T\zeta_{k-1})
     {\left(\zeta_{k-1}+w_{k-1}\right)}
   + O(\|\cdot\|^2)
   \big]
   \Big).
\end{align}

Substituting $e_{k-1}=\exp(\eta_{k-1})$ and using the BCH expansion


Neglecting deterministic higher-order terms gives the linearized discrete
error propagation
\begin{align}
\eta_k^-
= \eta_{k-1}
+ \Delta T\;
\operatorname{Ad}_{\widetilde g_{k-1}}\,
\Phi(-\Delta T\zeta_{k-1})\,w_{k-1}
\end{align}


Assume that by design
\begin{align}
L(y_k,\widetilde{y}_k) = K_k\left(H_{k}\,\left(\eta_{k-1}
+ \Delta T\;
\operatorname{Ad}_{\widetilde g_{k-1}}\,
\Phi(-\Delta T\zeta_{k-1})\,w_{k-1}\right)+z_{k}\right)
\end{align}
where
\begin{align}
\Gamma_\phi &:= \left.\frac{d}{d\varepsilon}\phi_{\exp(\varepsilon\xi)}(\gamma)\right|_{\varepsilon=0},\quad
J_L := \left.D_2 L(\gamma,\cdot)\right|_{\cdot=\gamma},\quad
H:=J_L \Gamma_\phi.
\end{align}


Then we have the linearization of the error propogation
\begin{align}
\eta_k&=A_{k-1}\eta_{k-1}+ G_{k-1}w_{k-1} - K_kH_k\eta_{k-1} - K_kH_kG_{k-1}w_{k-1} -K_kz_k\\
&=(A_{k-1}-\Delta TK_kH_k)\eta_{k-1}+(I-K_kH_k)G_{k-1}w_{k-1}-\Delta TK_kz_k
\end{align}
where
\begin{align}
A_{k-1}&= I\\
G_{k-1}&= \Delta T\,\operatorname{Ad}_{\widetilde{g}_{k-1}}\Phi(-\Delta T \zeta_{k-1})
\end{align}

**Note that these are in the exact form of the dicrete linear Kalman filter.**

---

### The DEKF

Prediction (state on the group and covariance):
\begin{align}
\widetilde g_k^- &= \widetilde g_{k-1}\exp{\left(\Delta T\left(\zeta_{k-1}+w_{k-1}\right)\right)}, \tag{48}\\
P_k^- &= A_{k-1} P_{k-1} A_{k-1}^{\top} + G_{k-1}\Sigma_qG_{k-1}^{\top}. \tag{51}
\end{align}
In (51), $\Sigma_q$ denotes the process noise covariance (same role as $\Sigma_p$ defined above).

Predicted output: consistent with the continuous-time relation $\widetilde y=\phi_{\widetilde g^{-1}}(\gamma)$, we use
\begin{align}
\widetilde y_k^- = \phi_{{\widetilde{g}_k^-}^{-1}}(\gamma). \tag{50}
\end{align}


Correction (group update and covariance update):
\begin{align}
K_k &= P_k^- H_k^{\top}\big(H_k P_k^- H_k^{\top} + \Sigma_m\big)^{-1}, \tag{51}\\
P_k &= \big(I - \Delta TK_k H_k\big)P_k^-, \tag{52}\\
\widetilde g_k &= \exp{\big(L(y_k,\widetilde y_k^-)\big)}\widetilde g_k^-, \tag{53}\\
\end{align}
These are the standard EKF gain and covariance update written with the intrinsic linearizations on $G$.  

### Rigidbody Kinematics

Rigid-body kinematics on $SE(3)$ are given by
\begin{align}
\dot g = g \cdot \zeta, \tag{54}
\end{align}
where $g\in SE(3)=SO(3)\rtimes\mathbb{R}^3$ and $\zeta\in\mathfrak{se}(3)$. Writing out the blocks,
\begin{align}
g &=
\begin{bmatrix}
R & o\\
0 & 1
\end{bmatrix},\qquad
\zeta =
\begin{bmatrix}
\widehat{\Omega} & V\\
0 & 0
\end{bmatrix},
\end{align}
with $R\in SO(3)$, $o,V\in\mathbb{R}^3$, and $\widehat{\Omega}\in\mathfrak{so}(3)$. The dot “$\cdot$” denotes ordinary matrix multiplication; $g$ represents the change of frame from a fixed frame $e$ to a body-fixed frame $b$.  

For $\zeta=(\Omega,V)$ and $\xi=(\Phi,U)$ in $\mathfrak{se}(3)$, the adjoint action is
\begin{align}
\operatorname{ad}_{\zeta} \xi = (\Omega\times\Phi, \Omega\times U - \Phi\times V),
\end{align}
so that, in block form,
\begin{align}
\operatorname{ad}\zeta =
\begin{bmatrix}
\widehat{\Omega} & 0\\
\widehat{V} & \widehat{\Omega}
\end{bmatrix}.
\end{align}

The left action $\phi:SE(3)\times\mathbb{R}^3\to\mathbb{R}^3$ is simple homogeneous multiplication:
\begin{align}
\phi_g(x) &=
\begin{bmatrix}
R & o\\
0 & 1
\end{bmatrix}
\begin{bmatrix}
x\\
\alpha
\end{bmatrix},
\quad
\text{with } g=(R,o)\in SE(3),\; x\in\mathbb{R}^3,\; \alpha\in\mathbb{R}.
\end{align}

Points $\gamma=[x\:\: 1]^\top$ represent a fixed physical point's homogeneous coordinates; the orbit $\phi_g(\gamma)$ collects its different perspectives. For $\zeta=(\Omega,V)$,
\begin{align}
(T_e\phi\circ \zeta)(\gamma) &=
\begin{bmatrix}
\widehat{\Omega} & V\\
0 & 0
\end{bmatrix}
\begin{bmatrix}
x\\
1
\end{bmatrix}.
\end{align}

A key term that appears in the intrinsic linearization is
\begin{align}
\phi_{\widetilde g^{-1}}\big( (T_e\phi\circ \operatorname{Ad}_{\widetilde g})\,\zeta \big)(\gamma)
&=
\zeta\widetilde g^{-1}\gamma\\
&=
\begin{bmatrix}
-\widetilde R^\top \widehat{(x-\widetilde o)}\widetilde R & I
\end{bmatrix}
\begin{bmatrix}
\Omega\\
V
\end{bmatrix},
\end{align}
with $\widetilde g=(\widetilde R,\widetilde o)$. This leads directly to the explicit structure used in $H_k$ for the SLAM example below.

### SLAM

Let $e$ be a fixed world frame and $p_i$ for $i=1,\dots,N$ be fixed landmarks. Write $\gamma_i=[x_i\:\:1]^\top\in\mathbb{R}^4$ for their homogeneous coordinates in $e$, and let $X_i$ denote the coordinates in the body frame related by $g\in SE(3)$. The SLAM model is
\begin{align}
\dot g = g\cdot(\zeta + n_p), \qquad
y_i = \phi_{g^{-1}}(\gamma_i) + n_m,
\end{align}
with $y_i=[X_i;1]^\top$, $n_p\sim\mathscr{N}(0,\Sigma_p)$, $n_m\sim\mathscr{N}(0,\Sigma_m)$. (Velocity biases can be added if needed.)

From the intrinsic linearization on $SE(3)$ one obtains the discrete EKF matrices (for sample time $\Delta T$):
\begin{align}
A_{k-1}
&=
\Bigl(I_{6\times 6} - \Delta T
\begin{bmatrix}
\widehat{\Omega}_{k-1} & 0\\
\widehat{V}_{k-1} & \widehat{\Omega}_{k-1}
\end{bmatrix}
\Bigr),
\\
H_k
&=
\begin{bmatrix}
\widetilde R_k^\top \,\widehat{(x_1-\widetilde o_k)}\,\widetilde R_k & -I_{3\times 3}\\
\widetilde R_k^\top \,\widehat{(x_2-\widetilde o_k)}\,\widetilde R_k & -I_{3\times 3}\\
\vdots & \vdots\\
\widetilde R_k^\top \,\widehat{(x_m-\widetilde o_k)}\,\widetilde R_k & -I_{3\times 3}
\end{bmatrix},
\qquad
G_{k-1} = \sqrt{\Delta T}\,I.
\end{align}
These plug into the intrinsic discrete EKF update
\begin{align}
\widetilde g_k^-&=\widetilde g_{k-1}\exp(\Delta T\,\zeta_{k-1}),\\
\widetilde g_k&=\widetilde g_k^- \exp{\big(\Delta T\,K_k(y_k-\widetilde y_k)\big)},
\end{align}
with the usual $K_k$, $P_k$ recursions defined earlier.

### Attitude Estimation from IMUs

In this case take $G=SO(3)$, $M=\mathfrak{g}\simeq\mathbb{R}^3$, and let the action be the adjoint $\phi=\mathrm{Ad}$ so that $\mathrm{Ad}_R x = Rx$ for $R\in SO(3)$, $x\in\mathbb{S}^2\subset \mathbb{R}^3$.

The Lie bracket is $ \mathrm{ad}_\Omega \Phi = \widehat{\Omega}\Phi = \Omega\times \Phi$. The measurement stacks two known inertial directions $e_1,e_2\in\mathbb{S}^2\subset \mathbb{R}^3$:
\begin{align}
y_k =
\begin{bmatrix}
R_k^{T} e_1\\
R_k^{T} e_2
\end{bmatrix}.
\end{align}
Using $\phi_{R^{T}}\circ (T_e\phi\circ \mathrm{Ad}_R)(\Omega)$, one finds the key identity (acting on any $x\in\mathbb{R}^3$):
\begin{align}
\big(\phi_{R^{T}}\circ T_e\phi\circ \mathrm{Ad}_R\cdot \Omega\big)x
= -(R^{T}\widehat{x}\,R)\Omega,
\end{align}
which yields the linearization used in $H_k$.  ￼

Discrete intrinsic EKF on $SO(3)$ (specialization of (48)–(53)):
\begin{align}
\widetilde R_k^{-} &= \widetilde R_{k-1}\exp{\big(\Delta T\left(\widehat{\Omega}_{k-1}+w_{k-1}\right)\big)}, \tag{55}\\
\widetilde R_k &= \widetilde R_k^{-}\exp{\Big(\Delta TK_k\big(y_k-\widetilde y_k^-\big)\Big)}, \tag{56}
\end{align}
with covariance prediction/gain update
\begin{align}
P_k^{-} &= A_{k-1} P_{k-1} A_{k-1}^{T} + G_{k-1}\Sigma_qG_{k-1}^{T}, \tag{57}\\
K_k &= P_k^{-} H_k^{T}\big(H_k P_k^{-} H_k^{T} + \Sigma_m\big)^{-1}, \tag{58}\\
P_k &= (I - K_k H_k)\,P_k^-. \tag{59}\qquad
\end{align}
and the discrete linearizations
\begin{align}
A_{k-1} = I_{3\times 3},\qquad
G_{k-1} = \Delta T\,\widetilde{R}_{k-1}\Phi(\Delta \widehat{\Omega}_{k-1}),\qquad
H_k =
\begin{bmatrix}
-\widehat{{\widetilde{R}_k^-}^{\top}e_1}\\
-\widehat{{\widetilde{R}_k^-}^{\top}e_2}
\end{bmatrix}.
\end{align}
The predicted output stacks the two directions in the estimated/body frame:
\begin{align}
\widetilde y_k^- =
\begin{bmatrix}
{\widetilde{R}_k^-}^{\top} e_1\\
{\widetilde{R}_k^-}^{\top} e_2\\
{\widetilde{R}_k^-}^{\top} e_3
\end{bmatrix}.
\end{align}
These equations are the attitude-only instance of the intrinsic discrete EKF on Lie groups specialized to $SO(3)$ with two vector measurements.  

##Simulation

### Estimating a Generated $R_k$

In [ ]:
import numpy as np
import plotly.graph_objects as go

# --- helpers (unchanged) ---
def sensor(dt, R, omega_body, sigma_omega=5e-3, sigma_dir=2e-2, rng=None):
    rng = rng or np.random.default_rng(0)
    e1 = np.array([1., 0., 0.])
    e2 = np.array([0., 1., 0.])
    e3 = np.array([0., 0., 1.])

    def n3(std):
        std = float(std) if np.isscalar(std) else np.asarray(std, float)
        if np.isscalar(std): std = np.full(3, std)
        return rng.normal(0.0, std, size=3)

    Omega_meas = omega_body + n3(sigma_omega)
    A_1_meas   = R.T @ e1 + n3(sigma_dir)
    A_2_meas   = R.T @ e2 + n3(sigma_dir)
    A_3_meas   = R.T @ e3 + n3(sigma_dir)
    # re-normalize to S^2
    A_1_meas /= max(np.linalg.norm(A_1_meas), 1e-12)
    A_2_meas /= max(np.linalg.norm(A_2_meas), 1e-12)
    A_3_meas /= max(np.linalg.norm(A_3_meas), 1e-12)
    return Omega_meas, A_1_meas, A_2_meas, A_3_meas

def innovation(R_minus: np.ndarray, A_1: np.ndarray, A_2: np.ndarray, A_3: np.ndarray):
    e1 = np.array([1.,0.,0.]); e2 = np.array([0.,1.,0.])
    e3 = np.array([0.,0.,1.])
    yhat_1 = R_minus.T @ e1
    yhat_2 = R_minus.T @ e2
    yhat_3 = R_minus.T @ e3
    # tangent-space residuals (each is 3x1)
    r_1 = np.cross(yhat_1, A_1)
    r_2 = np.cross(yhat_2, A_2)
    r_3 = np.cross(yhat_3, A_3)
    # stack (9,1)
    return np.hstack([r_1, r_2, r_3]).reshape(-1,1)

def angle_from_R(R):
    c = (np.trace(R) - 1.0) * 0.5
    c = float(np.clip(c, -1.0, 1.0))
    return np.arccos(c)  # radians



mr.set_sensor(sensor)
mr.set_KF_innovation(innovation)


In [ ]:
import numpy as np

def generate_constant_rate_trajectory(mr, dt=0.01, steps=20000, omega_body=None):
    """
    Generate a simple constant angular velocity trajectory on SO(3),
    compatible with run_offline_EKF_analysis().

    Parameters
    ----------
    mr : object
        RigidBodySim or equivalent class (needs .exp_map()).
    dt : float
        Time step [s].
    steps : int
        Number of integration steps.
    omega_body : ndarray(3,), optional
        Constant body angular velocity [rad/s].

    Returns
    -------
    trajectory : list of tuples
        Each element Xk = [ [R_true, o_dummy], omega_spatial, p_dummy, Xc_dummy ],
        compatible with run_offline_EKF_analysis().
    """

    if omega_body is None:
        omega_body = np.array([1.0, 1.0, 1.0])  # default constant rotation

    trajectory = []
    R_true = np.eye(3)
    p_dummy = np.zeros(3)
    Xc_dummy = np.zeros(3)
    o_dummy = np.zeros(3)

    for k in range(steps):
        # integrate body motion
        R_true = R_true @ mr.exp_map(dt * omega_body)
        # true spatial angular velocity
        omega_spatial = R_true @ omega_body
        # append full state (only first 2 used)
        trajectory.append([[R_true, o_dummy], omega_spatial, p_dummy, Xc_dummy])

    return trajectory


In [ ]:
trajectory=generate_constant_rate_trajectory(mr, dt=0.01, steps=20000, omega_body=None)

In [ ]:
mr.auto_tune_EKF(trajectory,1e-3*np.eye(3))

In [ ]:
sim_result=mr.run_offline_EKF_analysis(trajectory, dt=0.01, Sigma_q_factor=5., Sigma_m_factor=.10)

### Estimating Rigid Body Motion

In [ ]:
#Add the force model
def externalForceModel(qq,parameters,X):
    #Heavy Top Object
    M=parameters['M']; II=parameters['II']; g=parameters['g']; CM=parameters['CM'];
    R=X[0][0]; omega=X[1]; spi=R@ II@ R.T @ omega
    taue=1.*(-M*g*qq.hat_matrix(R@CM) @ [0,0,1])-0*spi;
    fe=np.array([0.,0.,0.]);
    return [taue,fe]

def actuator(qq,parameters, t, X, taue,fe):
    tauu=np.array([0,0,0]);
    fu=np.array([0,0,0]);
    return [tauu,fu]

mr.set_external_force_model(externalForceModel)
mr.set_actuator(actuator)

In [ ]:
DeltaT=0.01
cubeDimensions={'l':2.,'w':2.,'h':4.,'xp':0.,'yp':0.,'zp':0.,};
parameters={'dt':DeltaT,'CM':np.array([cubeDimensions['l']/2-cubeDimensions['xp'],cubeDimensions['w']/2-cubeDimensions['yp'],cubeDimensions['h']/2-cubeDimensions['zp']]), 'g':1, 'M':1, 'II':np.array([[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]])};
ICq=mr.q_from_axis_angles(np.pi/18,np.array([1,0,0])); ICR=mr.r_from_quaternions(ICq);
IComega=np.array([0.,0.,0.]); ICo=np.array([0.,0.,0.]); ICp=np.array([0.,0.,0.]);
ICXC=np.array([0,0,0]);
ICs=[[ICR,ICo],IComega,ICp,ICXC]
trajectory=mr.runga_kutta_method(DeltaT,20,parameters,ICs)

In [ ]:
aa4=mr.simulating_a_cube(DeltaT, 50., cubeDimensions, parameters,ICs)
fig1=mr.animated_cube_flat_shading(aa4,'Cube Pivoted at a Vertex')

In [ ]:
trajectory=mr.runga_kutta_method(DeltaT,15,parameters,ICs)

In [ ]:
import numpy as np
import plotly.graph_objects as go


# --- helpers (unchanged) ---
def sensor(dt, R, omega_body, sigma_omega=5e-3, sigma_dir=2e-2, rng=None):
    rng = rng or np.random.default_rng(0)
    e1 = np.array([1., 0., 0.])
    e2 = np.array([0., 1., 0.])
    e3 = np.array([0., 0., 1.])

    def n3(std):
        std = float(std) if np.isscalar(std) else np.asarray(std, float)
        if np.isscalar(std): std = np.full(3, std)
        return rng.normal(0.0, std, size=3)

    Omega_meas = omega_body + n3(sigma_omega)
    A_1_meas   = R.T @ e1 + n3(sigma_dir)
    A_2_meas   = R.T @ e2 + n3(sigma_dir)
    A_3_meas   = R.T @ e3 + n3(sigma_dir)
    # re-normalize to S^2
    A_1_meas /= max(np.linalg.norm(A_1_meas), 1e-12)
    A_2_meas /= max(np.linalg.norm(A_2_meas), 1e-12)
    A_3_meas /= max(np.linalg.norm(A_3_meas), 1e-12)
    return Omega_meas, A_1_meas, A_2_meas, A_3_meas

def innovation(R_minus: np.ndarray, A_1: np.ndarray, A_2: np.ndarray, A_3: np.ndarray):
    e1 = np.array([1.,0.,0.]); e2 = np.array([0.,1.,0.])
    e3 = np.array([0.,0.,1.])
    yhat_1 = R_minus.T @ e1
    yhat_2 = R_minus.T @ e2
    yhat_3 = R_minus.T @ e3
    # tangent-space residuals (each is 3x1)
    r_1 = np.cross(yhat_1, A_1)
    r_2 = np.cross(yhat_2, A_2)
    r_3 = np.cross(yhat_3, A_3)
    # stack (9,1)
    return np.hstack([r_1, r_2, r_3]).reshape(-1,1)

mr.set_sensor(sensor)
mr.set_KF_innovation(innovation)

In [ ]:
sim_result=mr.run_offline_EKF_analysis(mr.trajectory,dt=DeltaT, Sigma_q_factor=5., Sigma_m_factor=.10)

# References

[1] K. C. Wolfe, M. Mashner, and G. S. Chirikjian, “Bayesian fusion on Lie groups,” *Journal of Algebraic Statistics*, 2(1):75–97, 2011. [Link](https://rpk.lcsr.jhu.edu/publications/)

[2] S. Bonnable, P. Martin, and E. Salan, “Invariant extended Kalman filter: theory and application to a velocity-aided attitude estimation problem,” *Proc. 48th IEEE CDC/28th CCC*, pp. 1297–1304, Dec 2009. [Link](https://doi.org/10.1109/CDC.2009.5399990)

[3] S. Bonnabel, “Left-invariant extended Kalman filter and attitude estimation,” *Proc. 46th IEEE CDC*, pp. 1027–1032, Dec 2007. [Link](https://doi.org/10.1109/CDC.2007.4434662)

[4] G. Bourmaud, R. Mégret, A. Giremus, and Y. Berthoumieu, “Discrete extended Kalman filter on Lie groups,” *EUSIPCO 2013*, pp. 1–5, Sept 2013. [Link](https://hal.science/hal-00903252/document)

[5] G. S. Chirikjian, “Information theory on Lie groups and mobile robotics applications,” *Proc. 2010 IEEE ICRA*, pp. 2751–2757, May 2010. [Link](https://rpk.lcsr.jhu.edu/publications/)

[6] Y. Wang and G. S. Chirikjian, “Error propagation on the Euclidean group with applications to manipulator kinematics,” *IEEE Trans. Robotics*, 22(4):591–602, Aug 2006. [Link](https://ieeexplore.ieee.org/document/1673946)

[7] M. J. Piggott and V. Solo, “Stochastic numerical analysis for Brownian motion on SO(3),” *Proc. 53rd IEEE CDC*, pp. 3420–3425, Dec 2014. [Link](https://doi.org/10.1109/CDC.2014.7039919)

[8] O. Tuzel, F. Porikli, and P. Meer, “Learning on Lie groups for invariant detection and tracking,” *Proc. 2008 IEEE CVPR*, pp. 1–8, June 2008. [Link](https://doi.org/10.1109/CVPR.2008.4587521)

[9] A. Barrau and S. Bonnabel, “Intrinsic filtering on Lie groups with applications to attitude estimation,” *IEEE Trans. Automatic Control*, 60(2):436–449, Feb 2015. [Link](https://doi.org/10.1109/TAC.2014.2342911)

[10] S. Bonnabel and A. Barrau, “An intrinsic Cramér–Rao bound on SO(3) for (dynamic) attitude filtering,” *Proc. 54th IEEE CDC*, pp. 2158–2163, Dec 2015. [Link](https://dblp.org/rec/conf/cdc/BonnabelB15)

[11] A. Barrau and S. Bonnabel, “The invariant extended Kalman filter as a stable observer,” *IEEE Trans. Automatic Control*, 62(4):1797–1812, Apr 2017. [Link](https://doi.org/10.1109/TAC.2016.2594085)

[12] C. Lageman, J. Trumpf, and R. Mahony, “Gradient-like observers for invariant dynamics on a Lie group,” *IEEE Trans. Automatic Control*, 55(2):367–377, Feb 2010. [Link](https://doi.org/10.1109/TAC.2009.2034937)

[13] S. Bonnabel, P. Martin, and P. Rouchon, “Non-linear symmetry-preserving observers on Lie groups,” *IEEE Trans. Automatic Control*, 54(7):1709–1713, July 2009. [Link](https://doi.org/10.1109/TAC.2009.2020646)

[14] M. Izadi and A. K. Sanyal, “Rigid body attitude estimation based on the Lagrange–d’Alembert principle,” *Automatica*, 50(10):2570–2577, 2014. [Link](https://doi.org/10.1016/j.automatica.2014.08.010)

[15] S. Bonnabel and J. J. Slotine, “A contraction theory-based analysis of the stability of the deterministic extended Kalman filter,” *IEEE Trans. Automatic Control*, 60(2):565–569, Feb 2015. [Link](https://doi.org/10.1109/TAC.2014.2336991)
